<a href="https://colab.research.google.com/github/Chaddol/Bootcamp_project/blob/main/%EC%96%91%EC%B1%84%EB%A6%B0/oral_medication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### 약품식별 인공지능 개발을 위한 경구 약제 이미지 분류
##### 경구약제 약품식별 결과를 안내하고 피드백 할 수 있는 AI 서비스 모델이 요구되며 이를 위해 학습용 경구약제 이미지데이터 셋과 약품 정보를 분류 하는게 필요
##### 이미지 데이터 전처리 및 데이터 확인
##### 이미지 처리 기술을 활용하여 경구 약제 이미지 분류
#####Python, Pillow or Opencv, Deep learning
##### Git (1일 1 commit)